In [23]:
import os
import tweepy
from dotenv import load_dotenv

import json

#### Cargar llaves de la API
La función `load_dotenv` lee el archivo `.env` e inivializa las variables de ambiente que allí están. Luego se puede acceder a dichas variables de ambiente con el módulo `os`

In [24]:
load_dotenv()

consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]
bearer_token = os.environ["BEARER_TOKEN"]


client = tweepy.Client(
  bearer_token=bearer_token,
  consumer_key=consumer_key,
  consumer_secret=consumer_secret,
  access_token=access_token,
  access_token_secret=access_token_secret,
  wait_on_rate_limit=True
)

La siguiente función recibe las listas de cuentas y tweets estraídos inicialmente para ir agregando las cuentas y los tweets.

In [25]:
def extraer_cuentas_usuarios(lista_cuentas, lista_tweets, response):
    for i in range(len(response.includes['users'])):
        lista_cuentas.append(response.includes['users'][i])
        lista_tweets.append(response.data[i].text)

    print(len(lista_cuentas))

En esta parte es donde se usa el query para buscar Tweets, y luego usa la función anterior para extraer los Tweets que se encuentran en los responses de la paginación que hace Tweepy

In [26]:
# Search Recent Tweets
# This endpoint/method returns Tweets from the last seven days


query = 'depresión yo diagnosticado'
#query = 'depresión yo tengo'
#query = 'depresión yo tengo -lang:es #depresión #ansiedad'
#query = 'Petro'
#query = 'Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él'
#query = 'eafit'

# The method returns a Response object, a named tuple with data, includes,
# errors, and meta fields

cuentas_usuarios = []
tweets = []
for response in tweepy.Paginator(client.search_all_tweets,
                                query,
                                tweet_fields=['id', 'author_id', 'text'],
                                user_fields=['id', 'username', 'location'],
                                expansions='author_id',
                                start_time='2021-01-20T00:00:00Z',
                                end_time='2022-01-20T00:00:00Z',
                                max_results=100, limit=1):#.flatten(limit=100):
    extraer_cuentas_usuarios(cuentas_usuarios, tweets, response)



97


In [27]:
print(len(cuentas_usuarios))

97


In [ ]:
print(tweets)

In [ ]:
for usuario in cuentas_usuarios:
    print(usuario.location)

Luego, se verifica que los usuarios sean de Colombia. En caso contrario, no se toman en cuenta

In [21]:
# usuarios = []
# ids_usuarios = []
# for usuario in cuentas_usuarios:
#     if (usuario.location is not None):
#         #if ('Medellin' in usuario.location) or ('Medellín' in usuario.location) or ('medellin' in usuario.location) or ('medellín' in usuario.location):
#         if ('Colombia' in usuario.location) or ('colombia' in usuario.location):
#             usuarios.append(usuario.username)
#             ids_usuarios.append(usuario.id)

In [29]:
usuarios = []
ids_usuarios = []
for usuario in cuentas_usuarios:
    usuarios.append(usuario.username)
    ids_usuarios.append(usuario.id)

In [ ]:
print(usuarios)
print(ids_usuarios)

Crear el diccionario con el id que identifica a la persona y su correspondiente usuario

In [32]:
diccionario_ids_usuarios = dict(zip(ids_usuarios, usuarios))

#### Extraer tweets de usuarios
Con se extraen los tweets de los usuarios que se conservaron (las personas de Colombia). Sin embargo, se crea esta función previa ya que al momento de extraer retweets, cuando estos son muy largos (más de 240 caracteres) queda cortado el mensaje, por lo que realicé esta función, que va verificando si es retweet y en caso afirmativo extrae su texto completo que se encuenta la parte de `includes`

In [40]:
def extraer_tweets_usuarios(id, diccionario, response):
    print(response)
    exit()
    if id in list(diccionario.keys()):
        lista_tweets = diccionario[id]
    else:
        lista_tweets = []

    cont = 0
    for i in range(len(response.data)):
        if response.data[i]['referenced_tweets']:
            try:
                lista_tweets.append(response.includes['tweets'][cont].text)
                cont += 1
            except:
                lista_tweets.append(response.data[i].text)

        else:
            lista_tweets.append(response.data[i].text)

    diccionario[id] = lista_tweets

In [41]:
users_tweets = dict()
for id_usuario in ids_usuarios:
    for response in tweepy.Paginator(client.get_users_tweets,
                                    id_usuario,
                                    tweet_fields=['id', 'author_id', 'text'],
                                    user_fields=['id', 'username', 'location'],
                                    expansions='referenced_tweets.id',
                                    #expansions='author_id',
                                    max_results=100, limit=1):#.flatten(limit=100):
        extraer_tweets_usuarios(id_usuario, users_tweets, response)
        break

Response(data=[<Tweet id=1556113175493234688 text='@neverm0reee 🌚 https://t.co/5bQfbJPFGb'>, <Tweet id=1556110952298254336 text='@neverm0reee Te la tatuo , para que nunca me borres de tu vida https://t.co/yxfGAxTPL4'>, <Tweet id=1556110302550245376 text='@neverm0reee Con lag y todo luciendote mi ciela \U0001faf6 https://t.co/Dg2FvKi32s'>, <Tweet id=1556110124938235904 text='@TD_DAVICHO @MiniLeagueOP @TDelta_CR @Yerko_rojas_CR  manco tu momento a llegado'>, <Tweet id=1556109666429456384 text='@NellaBS_ Veni que aca estamos para ti mi diosa , 🥰'>, <Tweet id=1556105951031177217 text='@LatinMGirls @Fenix_LeagueBS @TeamWaikiPE @Jimuuu_ https://t.co/9opMH7agYm'>, <Tweet id=1556101404736909312 text='Ojo nuestra diosa stremeando y jugandl \nVengan a apoyarla https://t.co/KtS7n57Wnr'>, <Tweet id=1556084370171215873 text='@Alphaar0 La primera persona bkn que conoci y mantenemos vinculo es @P3rs3fon3e  \U0001faf6'>, <Tweet id=1556083834432651264 text='RT @nyurGG: Gracias a las chicas de @LatinMGi

KeyboardInterrupt: 

: 

In [35]:
print(users_tweets.keys())

dict_keys([4101389957, 2301666164, 1295261714376806400, 1253182381483347974, 1023767264934473729, 1256434052808077314, 796275180, 1236826748618043393, 134484328, 546505594, 1322218110221459462, 272157316, 1027751472132489216, 1418692959348961280, 186521527, 1471410537170976770, 1375830727045091333, 1425826061166604303, 407690120, 1363298289312014337, 1263713955966521350, 1461069161661034496, 903374147551723520, 1437217780017868803, 626095860, 1136044431348502528, 1247988677575311366, 977330247694725120, 893027243127123969, 1206915268557062145, 2827591822, 915999649802199040, 853291236287029248, 1450244697348263936, 1252289017, 1434600968826626048, 218026828, 747065730554859520, 868793483146002434, 1266552375705305089, 1729034707, 2429259504, 1448416109036703750, 1244663855583760386, 1165346915657105416, 1561724161, 213672616, 1027376756, 1414679081879224326, 1260747970301120512, 1213232508189298688, 728669870645235712, 1089638703318921216, 59939764, 195228252, 700392408, 10288115037916

In [36]:
print(list(users_tweets.keys())[0])
print(users_tweets[list(users_tweets.keys())[0]])

4101389957
['@KatNiniel tétrico pero me gusta\n\nacepto 😈 https://t.co/omn0XHekPy', '@KatNiniel te amo nini firmame una nalga', 'primero no me sale mi carl capitán y después me da lag me voy a cortar las venas https://t.co/FHTj9Qb2SH', 'Jugadores 🇨🇱, que quieran jugar la @MiniLeagueOP x @TDelta_CR tengo hasta el lunes, los leo', 'unas ganas d volver al compe', '𝗠𝗔𝗧𝗖𝗛𝗗𝗔𝗬 | #BRAWLSTARS \n\n𝗘𝗦𝗧𝗔𝗠𝗢𝗦 𝗘𝗡 𝗩𝗜𝗩𝗢! Disputando un lindo enfrentamiento en el canal de nuestra Diosa. 🔥\n\n🏆 @Fenix_LeagueBS\n🆚️ @TeamWaikiPE\n🕣 23:00hs 🇦🇷\n🎙️ @Jimuuu_\n🖥️ https://t.co/G0h7A3BW1D https://t.co/CyyovNuA1I', 'Menciona a la mejor persona que conociste gracias a brawl stars o algún medio relacionado a el.', 'Gracias a las chicas de @LatinMGirls por la pequeña entrevista! 👏🎙️\n\n@Jimuuu_ @1st_dai @card_png @neverm0reee 😊', '2x0 vs Waiki en la KDI League:)\nFeliz del progreso y de compartir con ustedes @card_png @1st_dai @neverm0reee @anderbs_ 🤞🏼❤️\ny de nuestro staff entero @LatinMGirls ✨️', '𝗠𝗔𝗧𝗖𝗛𝗗𝗔𝗬 | #BRAWL

In [37]:
print(len(users_tweets[list(users_tweets.keys())[0]]))

99


#### Guardar usuarios y Tweets en archivos

In [38]:
with open("tweets.json", "w") as fp:
    json.dump(users_tweets,fp)

In [39]:
with open("users.json", "w") as fp:
    json.dump(diccionario_ids_usuarios,fp)